In [13]:
import pandas as pd

def count_concerts_per_artist_venue(df):
    """Count the number of concerts per (artist, venue) for each year and month."""
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['concerts'] = 1

    # Group by artist, venue, year, month and sum the concerts
    grouped_df = df.groupby(['artist', 'venue', 'year', 'month'], as_index=False).agg({'concerts': 'sum'})

    return grouped_df

def reshape_to_wide_format(grouped_df):
    """Reshape the grouped DataFrame to a wide format."""
    pivot_df = grouped_df.pivot_table(index=['year', 'month'], 
                                       columns=['artist', 'venue'], 
                                       values='concerts', 
                                       fill_value=0)

    # Flatten the MultiIndex columns
    pivot_df.columns = [f'{artist}_{venue}' for artist, venue in pivot_df.columns]
    pivot_df.reset_index(inplace=True)

    return pivot_df
# Example DataFrame for testing
df = pd.DataFrame({
    'artist': ['Artist1', 'Artist2', 'Artist1', 'Artist3'],
    'venue': ['Venue1', 'Venue2', 'Venue1', 'Venue3'],
    'date': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-02-01', '2023-02-03'])
})

print(df)

    artist   venue       date
0  Artist1  Venue1 2023-01-01
1  Artist2  Venue2 2023-01-02
2  Artist1  Venue1 2023-02-01
3  Artist3  Venue3 2023-02-03


In [2]:
import pandas as pd
import os

def load_data(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    return pd.read_csv(file_path)

def count_concerts_per_artist_venue(df):
    # Ensure the 'date' column is in datetime format
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    
    # Drop rows where 'date' could not be converted
    df = df.dropna(subset=['date']).copy()
    print(df)  
    
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['concerts'] = 1
    print(df)
    return df.groupby(['artist', 'venue', 'year', 'month'], as_index=False).agg({'concerts': 'sum'})

def reshape_to_wide_table(concert_counts):
    unique_artists = concert_counts['artist'].unique()
    unique_venues = concert_counts['venue'].unique()
    
    # Create a cross product of artists and venues
    artist_venue_pairs = pd.MultiIndex.from_product([unique_artists, unique_venues], names=['artist', 'venue'])
    
    # Create a pivot table
    wide_table = concert_counts.pivot_table(index=['year', 'month'], 
                                            columns=['artist', 'venue'], 
                                            values='concerts', 
                                            fill_value=0)
    
    # Ensure all artist-venue pairs are included as columns
    wide_table = wide_table.reindex(columns=artist_venue_pairs, fill_value=0)
    
    # Return the wide table (year and month are already the index)
    return wide_table

def main():
    file_path = 'd:/CODING/CODING-WITH-GAURAVSVNITIAN/4.AI-ML/concert-analysis/data/concerts.csv'
    df = load_data(file_path)
    print(df)
    concert_counts = count_concerts_per_artist_venue(df)
    print(concert_counts)
    wide_table = reshape_to_wide_table(concert_counts)
    print(wide_table)

if __name__ == "__main__":
    main()

  artist venue        date
0      A     X  2024-01-10
1      B     Y  2024-01-15
2      A     X  2024-02-20
3      A     Y  2024-01-22
4      B     Y  2024-02-10
5      C     Z  2024-02-11
6      A     X      20/4/5
  artist venue       date
0      A     X 2024-01-10
1      B     Y 2024-01-15
2      A     X 2024-02-20
3      A     Y 2024-01-22
4      B     Y 2024-02-10
5      C     Z 2024-02-11
  artist venue       date  year  month  concerts
0      A     X 2024-01-10  2024      1         1
1      B     Y 2024-01-15  2024      1         1
2      A     X 2024-02-20  2024      2         1
3      A     Y 2024-01-22  2024      1         1
4      B     Y 2024-02-10  2024      2         1
5      C     Z 2024-02-11  2024      2         1
  artist venue  year  month  concerts
0      A     X  2024      1         1
1      A     X  2024      2         1
2      A     Y  2024      1         1
3      B     Y  2024      1         1
4      B     Y  2024      2         1
5      C     Z  2024      2    